In [2]:
import gc
import numpy as np

"""
화성-서울 네트워크 분석
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 1099개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 램프 전 본선 검지기
before_ramp = [73, 117, 126, 186, 220, 312, 344, 403, 414, 460]

# 램프 후 본선 검지기
after_ramp = [74, 119, 127, 188, 221, 313, 345, 404, 416, 461]

# 유입 검지기
input_ramp = [902, 904, 906, 908, 910]

#유출 검지기
output_ramp = [901, 903, 905, 907, 909]
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # 실측용량 C(2차로 4400)
    max_capacity = 4400
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]
    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before]
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after]
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)

    main_df = pd.concat(main_results, ignore_index=True)
    main_df = main_df.groupby("TimeGroup")[["Qm"]].mean().reset_index()


    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        q_in = (copy_df[copy_df["New_Measurement"] == input_]
                .groupby("TimeGroup").size().reset_index(name="q_in"))
        q_out = (copy_df[copy_df["New_Measurement"] == output_]
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        merged = q_in.merge(q_out, on="TimeGroup", how="outer").fillna(0)
        ramp_results.append(merged)

    ramp_df = pd.concat(ramp_results, ignore_index=True)
    ramp_df = ramp_df.groupby("TimeGroup")[["q_in", "q_out"]].mean().reset_index()

    rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="inner")

    # 램프 간섭 영향률 계산
    rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
    rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
    rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]

    return rfr_df

# 진출 원활율
def output_normality(original_df):

    copy_df = original_df.copy()
    copy_df["F(outrate)"] = 0.88
    return copy_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on=["TimeGroup"])
    .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    #modify_df = modify_df[(modify_df["StartTime"] >=3300) &(modify_df["StartTime"] < 6900)]
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([266, 531])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\테스트"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)
                #display("heavy_df : ", heavy_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)
                #display("entry_saturation_df : ", entry_saturation_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(rfr_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,62.150498,65.351045,61.876872,67.730463,63.053247,63.882715,67.339223,67.010082,66.031877,70.310229,...,72.115634,84.664520,86.540498,84.421516,72.499622,82.330672,88.056671,92.226598,90.848204,97.600158
2100~2400,71.442452,71.678391,82.444909,72.465017,81.426752,85.114888,72.428502,71.243685,72.497799,71.461513,...,96.965420,93.426634,88.682628,85.193274,91.726194,92.196864,91.269234,91.254858,93.035531,90.352588
2400~2700,72.306531,72.264891,71.240257,71.668595,71.258907,70.655259,71.103273,71.936624,71.480110,70.656084,...,70.299422,71.857407,86.996929,85.206073,81.900353,84.042830,83.393002,86.075057,86.036630,72.457765
2700~3000,89.352401,88.216680,89.899462,89.344077,91.362480,91.374365,93.504001,92.388655,93.155935,93.548455,...,93.510462,89.986986,84.162490,86.352242,86.364719,86.336244,82.290904,84.366745,87.015991,90.466332
3000~3300,68.659181,70.531914,68.428688,70.276516,62.228937,69.933561,66.270355,68.918734,67.919010,67.918568,...,95.850075,95.856953,94.120841,95.547004,95.824612,92.188279,94.656378,92.820068,94.247187,95.258895
3300~3600,83.431555,71.659223,82.805863,81.248875,84.175404,72.430403,81.250648,81.249133,72.045333,71.660081,...,91.160109,93.328686,96.232150,95.221762,94.301038,95.710516,94.697700,92.112597,91.846164,91.780615
3600~3900,71.306944,71.063272,71.057625,70.625536,71.304744,71.923867,71.434845,71.434816,71.863476,83.179804,...,86.481298,86.480560,86.483788,87.808809,84.438720,88.454953,90.691919,93.156217,90.101742,85.834608
3900~4200,72.033109,81.142882,80.394309,81.864505,82.580149,72.226096,71.840770,71.839965,71.263223,70.877342,...,92.641111,92.590961,93.298457,85.194836,85.635664,86.091806,86.604212,86.609035,85.380896,90.490255
4200~4500,63.158056,68.286059,63.162010,64.714200,61.971688,63.556353,66.882333,66.581355,67.269280,70.362459,...,81.148626,81.147077,71.824313,91.711628,92.607787,89.751742,82.178359,71.098895,69.408027,62.767141


환산점수_1.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,97.626554,97.798395,97.636072,97.564042,97.563198,97.547441,97.555131,97.533277,96.939433,96.482414,...,85.744520,82.733398,85.373748,87.768734,87.497315,82.218157,80.798019,72.253129,71.048332,70.510201
2100~2400,97.664674,97.772013,97.651483,97.605928,97.593641,97.588194,97.595456,97.572208,97.637840,97.644250,...,68.165900,68.588575,67.388080,67.780592,69.774069,71.172133,71.255725,71.737132,72.477327,82.120661
2400~2700,97.659767,97.615088,97.341159,97.068757,96.660716,96.321916,95.661856,96.320297,96.102953,95.790783,...,69.698935,68.645487,68.641544,65.410205,64.008904,64.656210,63.834243,64.630949,62.879879,60.899666
2700~3000,97.476094,97.569526,96.233844,95.601760,95.738895,95.979046,95.943764,95.303095,95.657041,95.947560,...,70.839292,71.105493,71.430015,71.166937,70.657297,70.056975,70.564631,70.328501,71.704006,70.773559
3000~3300,96.490217,97.578364,96.720334,95.989716,95.342190,94.977226,95.454809,96.192020,96.115436,95.789547,...,70.081964,71.196734,71.048939,72.185015,82.791942,85.750020,85.629915,85.567760,82.526624,84.733794
3300~3600,97.548385,97.715416,97.522389,96.951608,96.647138,97.045773,97.307264,97.128379,97.121634,97.424634,...,70.613467,70.181040,69.966950,65.349434,62.938395,59.481495,57.798868,56.431665,58.498752,57.488512
3600~3900,97.561564,97.624109,96.775631,96.785535,96.810741,96.369497,94.347817,87.910913,72.288378,69.948103,...,67.845577,68.217503,65.877517,67.238032,67.059724,70.112886,70.074353,70.570102,70.675940,71.029961
3900~4200,62.684250,64.447569,58.843853,56.827054,59.088848,58.249090,62.652272,66.152986,66.108714,71.459594,...,83.865286,82.008102,84.490077,83.085340,81.624073,80.762748,72.346452,71.278965,70.220370,69.935414
4200~4500,93.263964,89.549404,85.750160,86.037298,84.783294,84.350900,84.601971,90.490746,94.807338,94.267848,...,89.200316,88.805171,86.971591,93.524381,90.091683,92.868339,89.644120,89.707416,92.326573,92.128070


...

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,97.588502,97.722979,97.555712,97.456429,97.505063,97.509747,97.429915,97.260797,97.098695,97.103547,...,71.006924,71.730216,72.238450,80.849172,80.983183,83.702325,83.965867,84.736119,82.567820,82.448780
2100~2400,97.730149,97.808123,97.651275,97.629062,97.593691,97.589248,97.599014,97.608152,97.584545,97.619549,...,68.993656,66.483181,68.024990,70.274466,70.772281,70.656867,71.149113,71.818258,83.253494,82.557227
2400~2700,97.690776,97.763164,97.594549,97.541852,97.552347,97.578810,97.618449,97.615991,97.594183,97.561810,...,67.451099,69.866961,68.761456,68.610342,69.125206,68.953009,70.200601,70.198049,67.032665,66.937620
2700~3000,97.558230,97.641188,97.613005,97.609018,97.542284,97.502857,97.466654,97.127750,97.448833,97.505772,...,71.144200,70.347101,68.257408,65.707743,64.673193,62.153896,60.293825,57.111042,58.529702,59.334574
3000~3300,97.153765,97.138737,95.275984,93.726745,94.252760,93.835844,93.326159,94.102394,93.519376,92.541988,...,65.580264,68.700107,70.542062,70.451297,70.833561,71.726703,72.423500,81.433846,82.388537,80.872991
3300~3600,97.632380,97.763489,97.618344,97.578996,97.577705,97.582858,97.558354,97.545843,97.533982,95.889409,...,72.495598,72.492472,80.215205,80.715367,71.631927,70.393824,67.876861,67.328013,68.464028,70.494426
3600~3900,54.603885,58.995447,54.429452,54.872018,54.241967,54.351274,55.704180,54.117963,55.317823,65.078705,...,72.353346,71.212747,71.292907,71.033785,71.718921,80.976316,82.944842,83.126189,82.862217,83.883366
3900~4200,97.565155,97.574512,96.766337,96.359668,96.483431,96.244883,96.233508,96.865025,96.646072,95.676722,...,70.760887,71.222560,71.137039,71.738602,71.561688,71.297779,71.755646,71.620552,70.976091,70.728099
4200~4500,97.640196,97.809477,97.619237,97.554507,97.509543,97.516107,96.894416,96.928785,97.415191,97.370319,...,71.291815,71.534342,71.044509,71.170012,71.569081,71.825040,71.418983,71.734448,72.390292,71.748426


환산점수_3.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,97.711943,97.719387,97.694739,97.663883,97.658072,97.658762,97.650545,97.643025,97.652034,97.684246,...,86.159371,86.021220,85.024482,85.540502,85.916394,84.743555,83.901295,83.685391,87.441370,89.246853
2100~2400,95.634510,94.977829,95.481060,95.974518,96.248756,96.923074,97.349132,97.515008,97.195614,97.448044,...,68.031228,67.216145,65.982120,67.388436,68.519150,70.576723,71.053509,71.942795,71.952877,71.781570
2400~2700,97.601527,97.605663,97.604675,97.621306,97.594897,97.580509,97.540261,97.519156,97.530789,97.503956,...,89.732159,88.722063,88.963072,87.075299,87.061726,85.716866,86.712020,86.348992,82.845750,80.738289
2700~3000,97.591050,96.243193,91.010213,82.806790,70.985692,70.400248,71.011677,70.915830,70.913053,70.680220,...,92.155090,91.880098,92.026150,90.213227,88.808506,86.940104,83.889122,82.085585,80.538200,82.019720
3000~3300,70.723976,83.621393,86.983096,92.245464,95.975924,96.639928,96.683772,96.443579,96.607865,97.267311,...,72.184561,80.598588,81.298660,82.494517,81.882872,84.668820,87.636270,84.432042,87.453948,86.913437
3300~3600,97.811228,97.780410,97.706771,97.655590,97.656157,97.652033,97.683058,97.720235,97.708378,97.676081,...,70.962304,69.835017,70.567206,71.053585,71.831212,72.019934,72.283970,82.730989,82.837992,85.738412
3600~3900,97.848733,97.921201,97.800487,97.800758,97.800004,97.811644,97.778756,97.739767,97.740894,97.732451,...,95.707813,97.195468,96.199641,95.691127,93.863810,91.570657,90.436271,92.032913,93.190094,92.155471
3900~4200,97.706800,97.661721,95.441050,89.799685,72.197232,69.821521,70.445882,70.372040,70.086296,70.576003,...,88.067926,87.303347,88.829839,89.454063,90.815904,90.583649,89.169953,85.724019,83.336091,84.185287
4200~4500,54.571101,61.581133,66.202334,71.252412,83.243071,72.226393,70.390346,71.336493,71.210635,70.472285,...,97.517723,97.305043,96.794069,96.517723,96.429244,96.490559,97.327734,97.534349,97.587187,97.641772


환산점수_4.xlsx 생성 완료
